In [1]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
from sympy.abc import *
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
#%matplotlib notebook #incompatible with mpmath

Error importing optional module geopandas
Traceback (most recent call last):
  File "c:\Users\engin\anaconda3\lib\site-packages\_plotly_utils\optional_imports.py", line 30, in get_module
    return import_module(name)
  File "c:\Users\engin\anaconda3\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\engin\anaconda3\lib\site-packages\geopandas\__init__.py", line 3, in <module>
    from geopandas.geoseries import GeoSeries  # noqa
  File "c:\Users\engin\anaconda3\lib\site-package

This function queries the International Atomic Energy Agency's Nuclear Data Services (https://nds.iaea.org/).

API: https://nds.iaea.org/relnsd/vcharthtml/api_v0_guide.html 

Example Jupyter Notebook for automating data downloads: https://www-nds.iaea.org/relnsd/vcharthtml/api_v0_notebook.html  

You can only query for one type of decay at a time, among the following:

a: alpha decay 

bp: beta plus decay and electron capture

bm: beta minus decay

g: gamma emission

e: Auger and conversion electron

x: X-ray emission 

If the nuclide does not undergo that decay mode, the query will return a 0x1 dataframe with only the number 0

In [2]:
def lc_read_csv(url):
    '''
    Query the livechart service and return a pandas dataframe. 
    format: lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")
    '''
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))


parent, daughter = '238U', '238Np' 
test = ("https://www.nndc.bnl.gov/nudat3/getdecaydataset.jsp?nucleus=" +
         daughter.upper() + " &dsid=" + parent.lower() + "%20bM%20decay")
#get html from webpage
def look_up_decay_energy(html, search_string):
    '''
    This function takes the html from the webpage and returns the decay energy
    '''
    html = requests.get(test).text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        start_index = html.index(search_string) + len(search_string)
        after_html = html[start_index:]
        end_index = after_html.index(' ')
        return float(after_html[:end_index])
    except:
        print("error")
        return soup.text

#We can query the database for all of the nuclides at once
columns = ["mean_energy", "d_z", "d_n", "d_symbol", "intensity_beta", "half_life_sec"]
beta_decay_df = lc_read_csv(livechart + "fields=decay_rads&nuclides=all" + "&rad_types=bm")
beta_decay_df

,z,n,symbol,radius,unc_r,abundance,abundance_unc,energy_shift,energy,unc_e,...,unc_sp,binding,unc_ba,atomic_mass,unc_am,massexcess,unc_me,ENSDFpublicationcut-off,ENSDFauthors,Extraction_date
0,0,1,Nn,-0.1149,0.0027,,NaN,,0,,...,NaN,0,0,1008664.9159,47,8071.318060,44,31-Oct-2005,BALRAJ SINGH,2022-04-26
1,0,4,N,,NaN,,NaN,,0,,...,,,,,,NaN,,30-Aug-2017,J.E. Purcell and C.G. Sheu,2022-04-26
2,0,6,N,,NaN,,NaN,,0,,...,,,,,,NaN,,15-March-2017,J.H. Kelley and G.C. Sheu,2022-04-26
3,1,0,H,0.8783,0.0086,99.9855,78,,0,,...,0,0,0,1007825.031898,14,7288.971064,13,31-Oct-2005,BALRAJ SINGH,2022-04-26
4,1,1,H,2.1421,0.0088,0.0145,78,,0,,...,4,1112.2831,2,2014101.777844,15,13135.722895,15,1-May-2003,J.H. KELLEY and J.L. GODWIN,2022-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3362,116,177,Lv,,NaN,,NaN,,0,,...,869,7111,2,293204583,553,190568.000000,515,12-Aug-2005,M. GUPTA and THOMAS W. BURROWS,2022-04-26
3363,116,178,Lv,,NaN,,NaN,,0,,...,,,,,,NaN,,31-Jan-2019,BALRAJ SINGH,2022-04-26
3364,117,176,Ts,,NaN,,NaN,X,,,...,1090,7095,3,293208727,835,194428.000000,778,1-August-2010,M. GUPTA,2022-04-26
3365,117,177,Ts,,NaN,,NaN,,0,,...,785,7092,2,294210840,637,196397.000000,593,31-Jan-2019,BALRAJ SINGH,2022-04-26


In [3]:
beta_decay_df = lc_read_csv(livechart + "fields=decay_rads&nuclides=63ni32Si" + "&rad_types=bm")
beta_decay_df

,6


In [4]:
filtered_df = []
for row in range(beta_decay_df.shape[0]):
    try:
        filtered_df.append(beta_decay_df.iloc[row,:][columns])
    except:
        pass
filtered_df

[]

 This function takes a dataframe of parent nuclides and returns a list of dataframes of daughter nuclides

In [5]:
def get_daughter_df(parent_df):
    '''
    This function takes a dataframe of parent nuclides and returns a 
    list of dataframes of daughter nuclides
    '''
    daughter_nucleus= [str(int(parent_df['d_z'][row]) + int(parent_df['d_n'][row])) + 
                    ''.join(parent_df['d_symbol'][row]) for row in range(parent_df.shape[0])]
    daughter_df = [lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus[row].lower() 
                    + "&rad_types=bm") for row in range(parent_df.shape[0])]
    daughter_df = pd.concat(daughter_df)
    return daughter_df


It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0

In [29]:
#It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0
all_decay_types = ["a","bp","bm","g","e", "x" ]
def get_all_decay_types(isotope, decay_types = all_decay_types):
    '''
    This function takes an isotope and returns an array of dataframes of all of the decay types.
    Isotopes must be of the form "16O". 
    '''
    #I just want the halflife, 
    return [lc_read_csv(livechart + "fields=decay_rads&nuclides=" + isotope + 
            "&rad_types=" + decay_type) 
            if not lc_read_csv(livechart + "fields=decay_rads&nuclides=" + isotope + 
            "&rad_types=" + decay_type).shape == (0,1)
            else {decay_type: 0} for decay_type in decay_types]
get_all_decay_types("63Ni")

[{'a': 0},
 {'bp': 0},
    mean_energy  unc_me  intensity_beta  unc_ib  daughter_level_energy  \
 0       17.425   0.006             100       0                    0.0   
 
    max_energy  unc_me.1  log_ft unc_lf transition_type  ...  decay_%  unc_d  \
 0      66.946     0.008     6.7                      A  ...      100          
 
         q  unc_q d_z d_n  d_symbol ensdf_publication_cut-off  \
 0  66.977     15  29  34        Cu               28-Aug-2008   
 
                                     ensdf_authors  Extraction_date  
 0  HUO JUNDE and  YANG DONG and  HUO MEIRONG and        2022-07-01  
 
 [1 rows x 36 columns],
 {'g': 0},
 {'e': 0},
 {'x': 0}]

In [7]:
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=32Si&rad_types=bm")
df.iloc[ : , : 12]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame
0,69.55,0.11,100,0,0,227.2,0.3,8.21,0.06,A,158.1,0.18


In [8]:
df.iloc[ : , 12: 28]

,p_z,p_n,p_symbol,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,decay,decay_%,unc_d
0,14,18,Si,,0,,0+,157,,7,Y,4.954437e+09,2.208985e+08,B-,100,


In [9]:
df.iloc[ : , 28:]

,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,227.2,3,15,17,P,24-Aug-2011,CHRISTIAN OUELLET and BALRAJ SINGH,2022-07-01


In [21]:
float(df['mean_energy']) * float(df['intensity_beta']) / 100

KeyError: 'mean_energy'

In [ ]:
daughter_nucleus = str(int(df['d_z']) + int(df['d_n'])) + ''.join(df['d_symbol'])
#make all letters lowercase 
daughter_nucleus.lower()

'42k'

In [ ]:
daughter_df = lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus.lower() + "&rad_types=bm")
daughter_df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,21.41,0.21,0.07,0.01,3445.4,79.80,0.70,4.9800,0.0700,A,58.60,0.60,19,23,K
1,415.41,0.2,0.05,0.01,2424.3,1100.90,0.40,9.0500,0.0900,1NU,686.70,0.60,19,23,K
2,702.95,0.2,0.34,0.03,1837.2,1688.00,0.40,9.9200,0.0400,1U,986.49,0.36,19,23,K
3,824.32,0.17,17.64,0.09,1524.6,2000.60,0.30,7.5501,0.0023,1NU,1177.16,0.25,19,23,K
4,1565.86,,81.90,0.09,0.0,3525.22,0.18,9.4768,0.0006,1U,1961.70,0.09,19,23,K


In [ ]:
daughter_df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
1,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
2,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
3,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10
4,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-10


In [ ]:
daughter_df = get_daughter_df(df)

In [ ]:
third_gen_df = get_daughter_df(daughter_df)
third_gen_df

,0


In [31]:
a = lc_read_csv(livechart + "fields=decay_rads&nuclides=42ar&rad_types=bm")
lc_read_csv(livechart + "fields=decay_rads&nuclides=238pu&rad_types=a")

,energy,unc_en,intensity,unc_i,daughter_level_energy,hindrance_factor,unc_hf,p_z,p_n,p_symbol,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,4430.70,0.4,0.0000011,,1085.26,3.8,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
1,4470.80,0.3,0.0000012,0.0000002,1044.536,7.3,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
2,4491.10,0.3,,,1023.9,,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
3,4524.90,0.4,0.00000013,,989.430,179,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
4,4565.80,0.3,0.00000025,0.00000008,947.64,192,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
5,4579.00,,0.00002,,,,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
6,4590.00,,0.000012,,926.720,5.7,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
7,4661.00,,0.00000593,0.00000023,851.74,42,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
8,4662.60,0.4,0.00000009,0.00000004,849.266,2850,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11
9,4704.00,,0.00005,,809.907,10,,94,144,Pu,...,100,,5593.27,19,92,142,U,1-Jun-2006,E. BROWNE and J. K. TULI,2022-05-11


Where there 1 is one mole of the 0-th generation nucleus at $t=0$ there are $m_i$ moles of the $i-th$ generation nucleus at time $t$. We assume that the $i-th$ generation nucleus has a measurable e-folding time of $\lambda_i$. Additionally, $r_i$ is the decay rate of the $i-th$ generation nucleus in moles/time.
\begin{equation}
\begin{split}
\Large m_0 = e^{\Large\frac{t}{-\lambda_0}}\\
\Large m_{i>0} = -\frac{1}{\lambda_0}\int_0^te^{\Large t(\Large\frac{1}{ -\lambda_0}+\frac{1}{-\lambda_1})}a_{i-1}dt\\
\Large r_i = -\frac{1}{\Large\lambda_0}e^{\Large\frac{t}{-\lambda_0}}a_i\\
\Large a_i = \prod_{k=1}^{i}(1-e^{ t\Large\frac{1}{-\lambda_k}})
\end{split}
\end{equation}
We also assume an average decay energy $\bar{E}_i$ for the nuclide in each generation, and thus we have for the total power produced by the decay chain and for the power density $\rho_P$
\begin{equation}
\begin{split}
\Large P = \sum_{j=0}^N \bar{E}_ir_i\\
\Large \rho_P = \frac{\sum_{j=0}^N \bar{E}_ir_i}{\sum_{j=0}^N m_i}
\end{split}
\end{equation}
Less than $10^{-7}$ of the mass of the nuclide is lost in $\beta-$ decay so we have
\begin{equation}
\begin{split}
\Large \rho_P = \frac{1}{m_0}\sum_{j=0}^N \bar{E}_ir_i\\
\end{split}
\end{equation}

In [ ]:
#Quickly calculating the decay rate of the i-th generation nuclide
def formulate_decay_rate(e_folding_times):
    '''
    The e-folding times must be in a numpy array.
    Returns a formula for the decay rate of each generation. 
    '''
    exponent_array = -1 / e_folding_times
    decay_rates = len(exponent_array)  * [sym.N(0)]
    decay_rates[0] = sym.exp(t * exponent_array[0]) * exponent_array[0]
    for index, L in enumerate(exponent_array[1:]):
        decay_rates[index+1] = decay_rates[index] * (1 + sym.exp(t * L) * L) 
    return decay_rates

def eval_decay_rates(decay_rates, time_array):
    '''
    This function takes the formula for the decay rate of each generation
    and substitutes each value in the time array for t.
    Each decay_rate must be a sympy expression. 
    https://docs.sympy.org/ 
    Rewrite so that the outer loop is over the time array and the inner loop
    is over the generations. This can be done by using (suggested by copilot: 
    the np.meshgrid function) or a 2d array (my first thought)
    '''
    try: #will only evaluate if decay_rates is an array of sympy expressions
        evaluated_decay_rates = [np.array([formula.subs(t, time) for formula in decay_rates])
                                    for time in time_array]
    except:
        evaluated_decay_rates = [decay_rates.subs(t, time) for time in time_array]
    return evaluated_decay_rates


#Calculating the power density of a decay chain
def calc_power_density(decay_rates, decay_energies, initial_mass):
    '''
    Rewrite such that is sums the entire chain and NOT across time
    decay_rates must be a numpy array in moles/second
    decay_energies must be a numpy array in keV/decay
    This function takes the decay rates and energies of the decay chain.
    Returns the power density in watts/g
    '''
    power_density =  np.array([sum(decay_rate * decay_energies[generation]) 
                        for generation, decay_rate in enumerate(decay_rates)])
    #convert W/g
    power_density *=  units.keV.to(units.J) * float(constants.N_A * units.mol) / initial_mass
    return power_density


In [ ]:
e_folding_times = np.array([10**2, 10])
time_array = np.linspace(0, 10**2, 10**2)
decay_rates = formulate_decay_rate(e_folding_times)
evaluated_decay_rates = eval_decay_rates(decay_rates, time_array)
decay_energies = np.array([1 / d for d in evaluated_decay_rates])
power_densities = calc_power_density(evaluated_decay_rates, decay_energies, 1)

2


\begin{equation}
\begin{split}
m = 2^{-t\frac{1}{h}}\\
m = e^{-t\frac{1}{h}\log2}\\
m = e^{-t\frac{1}{\lambda}}\\
-t\frac{1}{h}\log2 = -t\frac{1}{\lambda}\\
h = \lambda\log2
\end{split}
\end{equation}

In [85]:
def lambda_to_half_life(e_folding_time):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return np.log(2) * e_folding_time

def half_life_to_lambda(half_life):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return half_life / np.log(2)

pu238t = half_life_to_lambda(87.7 * units.year.to(units.s))
ni63t = half_life_to_lambda(101.2 * units.year.to(units.s))

def exact_power_density(e_folding_time, molar_mass, avg_decay_energy):
    return (avg_decay_energy * constants.N_A * units.mol / 
            (e_folding_time * molar_mass * 2)).to(units.W / units.g)

#exact_power_density(half_life_to_lambda(101.2 * units.year), 63*units.g, 17 * units.keV)
#What would make the best RTG? I need a shielding estimate :/. For now we will ignore shielding. 
exact_power_density(half_life_to_lambda(71.1 * units.year), (84+64)*units.g, 3182.69 * units.keV)

<Quantity 0.32049098 W / g>

In [90]:
def decay_chain_df(nuclide, columns = None):
    '''
    Thus function takes a nuclide and returns the decay chain of that nuclide. 
    The decay chain will be in the form of a dataframe with the following 
    columns: e-folding time (in seconds), decay rate (in moles/second),
    and decay energy (in keV/decay). 
    This will later be used to plot the decay chain's total power density 
    as a function of time.    
    Columns are the columns to include in each dataframe. 
    '''
    try:
        df_list = [lc_read_csv(livechart + "fields=decay_rads&nuclides=" +
                    nuclide.lower() + "&rad_types=bm")[columns]]
        df_list[-1]["e_folding_time_sec"] = half_life_to_lambda(df_list[-1]["half_life_sec"])
    except: #the nuclide isn't in the database
        return 
    if (columns):
            while len(df_list[-1]) > 0:
                print("Getting daughter nuclides...")
                try:
                    df_list.append(get_daughter_df(df_list[-1])[columns])
                    df_list[-1]["e_folding_time_sec"] = half_life_to_lambda(df_list[-1]["half_life_sec"])
                except:
                    #The daugher nuclide is not in the livechart database or is stable.
                    print("exception thrown")
                    return df_list
    else:
        while len(df_list[-1]) > 0:
            print("Getting daughter nuclides...")
            df_list.append(get_daughter_df(df_list[-1]))
            df_list[-1]["e_folding_time_sec"] = half_life_to_lambda(df_list[-1]["half_life_sec"])
    if (len(df_list) > 1):
        df_list = df_list[:-1]
    return df_list

nuclides_list = pd.read_csv(subprocess.os.getcwd() + '\\nuclide_list.csv')
columns = ["mean_energy", "d_z", "d_n", "d_symbol", "intensity_beta", "half_life_sec"]
all_decay_chains = [decay_chain_df(n, columns = columns) for n in nuclides_list.iloc[:,0]]

Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
Getting daughter nuclides...
exception thrown
Getting daughter nuclides...
Getting daughter nuclides...
exception throw

In [129]:
eval_r_i = sym.lambdify(t, calc_decay_rate(1, np.array([10**9.5])))
time_array = np.logspace(-7, 2, 10**3)
decay_rates = np.absolute([(np.round(eval_r_i(t), 10)) for t in time_array]) * units.s.to(units.year)
xlabel, ylabel = "Time (years)", "Decay Rate (moles/seconds)"
decay_df = pd.DataFrame({xlabel: time_array, ylabel: decay_rates})
px.scatter(decay_df, x = xlabel, y = ylabel, log_x = True, log_y = True,
            title = "Decay Rate of chain with e-folding times " + str(e_folding_times))

In [131]:
time_array

array([1.00000000e-07, 1.02096066e-07, 1.04236067e-07, 1.06420924e-07,
       1.08651577e-07, 1.10928986e-07, 1.13254132e-07, 1.15628013e-07,
       1.18051653e-07, 1.20526094e-07, 1.23052400e-07, 1.25631660e-07,
       1.28264983e-07, 1.30953502e-07, 1.33698374e-07, 1.36500781e-07,
       1.39361927e-07, 1.42283046e-07, 1.45265393e-07, 1.48310251e-07,
       1.51418933e-07, 1.54592774e-07, 1.57833141e-07, 1.61141428e-07,
       1.64519059e-07, 1.67967487e-07, 1.71488197e-07, 1.75082703e-07,
       1.78752553e-07, 1.82499324e-07, 1.86324631e-07, 1.90230119e-07,
       1.94217468e-07, 1.98288395e-07, 2.02444651e-07, 2.06688025e-07,
       2.11020343e-07, 2.15443469e-07, 2.19959307e-07, 2.24569800e-07,
       2.29276931e-07, 2.34082728e-07, 2.38989257e-07, 2.43998630e-07,
       2.49113003e-07, 2.54334576e-07, 2.59665597e-07, 2.65108360e-07,
       2.70665207e-07, 2.76338529e-07, 2.82130768e-07, 2.88044415e-07,
       2.94082017e-07, 3.00246171e-07, 3.06539530e-07, 3.12964801e-07,
      

In [ ]:
p_t = 1 - sym.exp(-1 * t / lambda_t)
prob_not_decay = sym.lambdify(([t, lambda_t]), p_t)
def daughter_nucleus_abundance(t, lambda_d, parent_initial_mol, parent_nucleus_decay):
    m_0 = parent_initial_mol
    p = 1 - sym.exp(-1 * t / lambda_d)
    delta_tau = sym.integrate(p * parent_nucleus_decay, (t, 0, t))
    evaluate_delta_tau = sym.lambdify(([tau, lambda_m, lambda_d, m_0]), delta_tau, 'numpy')
    t, lambda_d, parent_initial_mol = np.float(t), np.float(lambda_d), np.float(parent_initial_mol)
    return evaluate_delta_tau(t, lambda_d, m_0, parent_nucleus_decay)

